## 합성곱 신경망

### 파이토치로 패션 MNIST CNN 실습
- 텐서플로우(케라스)로 CNN을 하든, 파이토치로 CNN을 하든 차이가 없음

#### 데이터셋 불러오기

In [7]:
# 사용 모듈 로드
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

In [8]:
# 이전에는 sklearn에 있는 StandardScaler 사용
# 정규화 및 텐서 변환
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))     # Fashion-MNIST 자주 사용하는 정규화 범위
])

# Fashion-MNIST를 Dataset으로 변경
## keras는 C:\Users\Admin\.keras\datasets
## PyTorch는 현재위치에 다운로드
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform)

# Dataset을 DataLoader로 변경
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

100.0%
100.0%
100.0%
100.0%


#### 클래스 레이블

|레이블|0|1|2|3|4|5|6|7|8|9|
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|패션MNIST|티셔츠/탑|바지|스웨터/Pullover|드레스|코드|샌달|셔츠|스니커즈|가방|앵클 부츠|